# ✅Step 3: Data Cleaning

This notebook will clean up the posts.json file by:
1. Removing posts with non-English titles
2. Removing irrelevant columns
3. Converting relevant values to the correct data types

We will then use GPT-3.5 to extract a list of food ingredients from the recipe comment.

## 0. 🎯Setup

In [7]:
import sys

import pandas as pd
import tabulate
from datetime import datetime

from tqdm import tqdm
tqdm.pandas()

# Import our own modules
sys.path.append("../scripts/")
import chadtools

### 0.1. Load json file 

Load dataframe from json file generated from [02 Scraping OP Comments](./02%20Scraping%20OP%20comment.ipynb)


In [36]:
df = pd.read_json('../data/posts_with_comments.json', orient='records')
df.shape

(2066, 119)

## 1. 🧹Simple Data Cleanup

Some posts are not formatted properly or have been deleted. We will remove them from our dataframe by checking for newlines, which are present in all properly formatted recipes.

In [37]:
df = df[df['ingredient_comment'].str.contains("\n") == True]

df.shape

(1747, 119)

We clean up the dataframe by filtering for the columns that we want.

In [38]:
desired_columns = ['id',
                   'title',
                   'score',
                   'num_comments',
                   'ingredient_comment',
                   'created_utc', 
                   'upvote_ratio',
                   'link_flair_text',
                   'author',
                   'url',
                   'comment_link',
                   'permalink']

df_filtered = df.loc[:, desired_columns]
df_filtered.head()

,id,title,score,num_comments,ingredient_comment,created_utc,upvote_ratio,link_flair_text,author,url,comment_link,permalink
0,1ah8m5s,Thai Green Curry Chicken Satay,41,1,You could use any curry paste you like. Do you...,1706893611,0.87,Recipe,butchec,https://i.redd.it/g5xtfz2ve7gc1.jpeg,https://oauth.reddit.com/r/recipes/comments/1a...,https://reddit.com/r/recipes/comments/1ah8m5s/...
1,1ag9jx2,Potato Pavé,252,7,A French side dish that goes well with anythin...,1706789454,0.97,Recipe,TimSumrall,https://i.redd.it/1fy6x4t3tyfc1.jpeg,https://oauth.reddit.com/r/recipes/comments/1a...,https://reddit.com/r/recipes/comments/1ag9jx2/...
2,1afs8r5,Pretzels 🥨,116,9,Yield: 8 medium pretzels\n\nSee how I made the...,1706734068,0.94,Recipe,shushyum,https://i.redd.it/v8e6vvxh8ufc1.jpeg,https://oauth.reddit.com/r/recipes/comments/1a...,https://reddit.com/r/recipes/comments/1afs8r5/...
3,1afoma3,Sweet and Sour Tofu,54,4,I love figuring out ways to make tofu deliciou...,1706725321,0.89,Recipe,parisrosaries,https://i.redd.it/9nhm0k2mitfc1.jpeg,https://oauth.reddit.com/r/recipes/comments/1a...,https://reddit.com/r/recipes/comments/1afoma3/...
4,1aez4r2,Pork Tenderloin Sous Vide,28,4,**Ingredients:**\n\n**Tenderloin**\n\n* 1 Pork...,1706648218,0.69,Recipe,hoosyourdaddyo,https://i.redd.it/tq0z0nkd5nfc1.jpeg,https://oauth.reddit.com/r/recipes/comments/1a...,https://reddit.com/r/recipes/comments/1aez4r2/...


In [39]:
df_filtered.shape

(1747, 12)

We filter out posts with empty rows.

In [40]:
important_cols = ['id', 'title', 'ingredient_comment', 'permalink']
df_filtered.dropna(axis=0, subset=important_cols, inplace=True)
df_filtered.shape

(1747, 12)

### 1.1 Filter out posts with non-english titles

In [41]:
# filter the english posts by applying custom function
df_filtered = df_filtered[df_filtered['title'].progress_apply(chadtools.is_english, threshold_rank=5)]

df_filtered.shape

100%|██████████| 1747/1747 [00:03<00:00, 509.91it/s]


(1633, 12)

### 1.2 Convert data to more appropriate types 

In [42]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1633 entries, 0 to 2065
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1633 non-null   object 
 1   title               1633 non-null   object 
 2   score               1633 non-null   int64  
 3   num_comments        1633 non-null   int64  
 4   ingredient_comment  1633 non-null   object 
 5   created_utc         1633 non-null   int64  
 6   upvote_ratio        1633 non-null   float64
 7   link_flair_text     1633 non-null   object 
 8   author              1633 non-null   object 
 9   url                 1633 non-null   object 
 10  comment_link        1633 non-null   object 
 11  permalink           1633 non-null   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 165.9+ KB


We will convert the following columns to the following types:
- `title` to string
- `score` to int32
- `num_comments` to int16
- `created_utc` to datetime
- `upvote_ratio` to float16

#### 1.2.1 Convert created_utc to a datetime object

In [43]:
df_filtered['created_utc'] = df_filtered['created_utc'].apply(lambda x: datetime.fromtimestamp(x))

Starting from 31 Aug 2020, r/recipes imposed strict rules on the format of new posts, which made the format substantially more consistent. Hence, we will only keep posts from 31 Aug 2020 onwards for ease of data collection.

In [44]:
cutoff_datetime = pd.to_datetime("2020-08-31 00:00:00")

# Filter out rows where 'created_utc' is before the cutoff datetime
df_filtered = df_filtered[df_filtered['created_utc'] >= cutoff_datetime]

df_filtered.tail()

,id,title,score,num_comments,ingredient_comment,created_utc,upvote_ratio,link_flair_text,author,url,comment_link,permalink
1985,jhi1cz,Vegetarian Pumpkin Meatballs with a Heart of C...,54,6,\n\nHere another recipe with pumpkin!! A deli...,2020-10-24 23:22:15,0.89,Fruit\Vegetarian,italian_cook,https://www.reddit.com/gallery/jhi1cz,https://oauth.reddit.com/r/recipes/comments/jh...,https://reddit.com/r/recipes/comments/jhi1cz/v...
1986,jcgb7j,Bitter gourd yogurt curry....with no bitternes...,6,6,Recipe.....\n\n[Short Video](https://youtu.be/...,2020-10-16 20:18:12,0.62,Fruit\Vegetarian,PassionateHobbies,https://i.redd.it/bpootodgbit51.jpg,https://oauth.reddit.com/r/recipes/comments/jc...,https://reddit.com/r/recipes/comments/jcgb7j/b...
1988,iz12pg,Ottolenghi's Baked Orzo w/Mozzarella,22,5,Ingredients:\n\n* 7 Tablespoons olive oil\n* ...,2020-09-24 17:59:05,0.83,Fruit\Vegetarian,BrinaElka,https://i.redd.it/l7osuhkcm4p51.jpg,https://oauth.reddit.com/r/recipes/comments/iz...,https://reddit.com/r/recipes/comments/iz12pg/o...
1989,iw3wli,Mushroom Barley Stew with Crispy Oyster Mushrooms,2693,41,**Recipe here originally:** [**Easy Mushroom B...,2020-09-20 01:27:07,0.98,Fruit\Vegetarian,BushyEyes,https://i.redd.it/511qxuct57o51.jpg,https://oauth.reddit.com/r/recipes/comments/iw...,https://reddit.com/r/recipes/comments/iw3wli/m...
1990,isunwt,Easy Tomato Risotto with Parmesan,1796,35,**Recipe here originally:** [**Easy Tomato Ris...,2020-09-14 22:32:27,0.99,Fruit\Vegetarian,BushyEyes,https://i.redd.it/0qb76yy3m6n51.jpg,https://oauth.reddit.com/r/recipes/comments/is...,https://reddit.com/r/recipes/comments/isunwt/e...


#### 1.2.2 Convert the more primitive types to more size-efficient types.

In [45]:
df_filtered['score'] = df_filtered['score'].astype('int16')
df_filtered['num_comments'] = df_filtered['num_comments'].astype('int16')   
df_filtered['upvote_ratio'] = df_filtered['upvote_ratio'].astype('float16')

In [46]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 998 entries, 0 to 1990
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  998 non-null    object        
 1   title               998 non-null    object        
 2   score               998 non-null    int16         
 3   num_comments        998 non-null    int16         
 4   ingredient_comment  998 non-null    object        
 5   created_utc         998 non-null    datetime64[ns]
 6   upvote_ratio        998 non-null    float16       
 7   link_flair_text     998 non-null    object        
 8   author              998 non-null    object        
 9   url                 998 non-null    object        
 10  comment_link        998 non-null    object        
 11  permalink           998 non-null    object        
dtypes: datetime64[ns](1), float16(1), int16(2), object(8)
memory usage: 83.8+ KB


As seen, the size of the dataframe has been reduced by **50%**.

## 2. 🎯Extract Ingredients

We will use `regex` to extract the ingredients portion of the comment, and disregard the cooking instructions for simplicity. We will then use GPT-3.5 to identify the ingredients.

### Filter out ingredients portion of the comments

In [47]:
df_filtered['ingredient_comment_truncated'] = df_filtered['ingredient_comment'].progress_apply(chadtools.extract_ingredients_text)

100%|██████████| 998/998 [00:00<00:00, 24341.95it/s]


### Set up GPT Client

In [48]:
my_client = chadtools.setup_gpt_client()

In [49]:
df_filtered['ingredient_comment_truncated'].head()

0    You could use any curry paste you like. Do you...
3    I love figuring out ways to make tofu deliciou...
4    **Ingredients:**\n\n**Tenderloin**\n\n* 1 Pork...
5    [Recipe Link](https://www.sarahfreia.com/blog/...
7    See how I made them [here](https://youtu.be/W8...
Name: ingredient_comment_truncated, dtype: object

### Use GPT to identify ingredients

WARNING: This step calls the OpenAI GPT API, which incurs costs. Ensure sufficient tokens are available before running this step.

In [50]:
df_filtered['gpt_ingredients'] = df_filtered['ingredient_comment_truncated'].progress_apply(chadtools.get_ingredient_list, client=my_client)
df_filtered['gpt_ingredients'].head()

100%|██████████| 998/998 [31:13<00:00,  1.88s/it]


0    [chicken satay, homemade green curry paste, ve...
3    [tofu, ground black pepper, salt, cornstarch, ...
4    [tenderloin, salt, fresh ground pepper, smoked...
5    [butter, brown sugar, white sugar, vanilla ext...
7    [milk, eggs, water, salt, flour, oil, ground b...
Name: gpt_ingredients, dtype: object

### Reorder, filter, save as JSON

Finally, filter and reorder the columns, and save the dataframe as a JSON file for use in the next step.

In [51]:
ordered_cols = ["id", "title", "gpt_ingredients", "ingredient_comment_truncated", "score", "upvote_ratio", "link_flair_text", "author", "created_utc", "url", "permalink"]
df_filtered = df_filtered.loc[:, ordered_cols]
df_filtered.to_json('../data/cleaned_posts_with_ingredient_list.json', orient='records', indent=4)